In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)

In [ ]:
import numpy as np

In [ ]:
from tda.experiments.lid.lid_binary import logger, run_experiment, Config, Architecture, get_feature_datasets
from tda.models.architectures import get_architecture, mnist_lenet
from tda.models import get_deep_model
from tda.models.datasets import Dataset

In [ ]:
config = Config(
    epochs=50,
    dataset="MNIST",
    architecture=mnist_lenet.name,
    train_noise=0.0,
    nb_batches=1,
    attack_type="FGSM",
    noise=0.0,
    num_iter=1,
    batch_size=100,
    number_of_nn=20,
    successful_adv=True
)

In [ ]:
dataset = Dataset(name=config.dataset)

logger.info(f"Getting deep model...")
archi: Architecture = get_deep_model(
    num_epochs=config.epochs,
    dataset=dataset,
    architecture=get_architecture(config.architecture),
    train_noise=config.train_noise
)

datasets = dict()

for epsilon in [0.01, 0.025, 0.05, 0.1, 0.4, 1.0]:
    ds_train, ds_test = get_feature_datasets(
        config=config,
        epsilon=epsilon,
        dataset=dataset,
        archi=archi
    )
    datasets[epsilon] = (ds_train, ds_test)

In [ ]:
datasets[0.4][0]

In [ ]:
linfs = dict()
l2s = dict()

for key in datasets:
    linf_mean_train = datasets[key][0].query("label > 0").linf_norm.mean()
    l2_mean_train = datasets[key][0].query("label > 0").l2_norm.mean()
    
    linfs[key] = linf_mean_train
    l2s[key] = l2_mean_train

## Supervised Learning

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

scaler = StandardScaler()

all_aucs = dict()
all_coefs = dict()

for epsilon in datasets:
    train_ds, test_ds = datasets[epsilon]
    
    detector = LogisticRegression(
        fit_intercept=True,
        verbose=1,
        tol=1e-5,
        max_iter=1000,
        solver='lbfgs'
    )
    
    ds_train_scaled = scaler.fit_transform(train_ds.iloc[:, :-3])

    detector.fit(X=ds_train_scaled, y=train_ds.iloc[:, -3])
    coefs = list(detector.coef_.flatten())
    logger.info(f"Coefs of detector {coefs}")
    
    ds_test_scaled = scaler.transform(test_ds.iloc[:, :-3])

    test_predictions = detector.predict_proba(X=ds_test_scaled)[:, 1]
    auc = roc_auc_score(y_true=test_ds.iloc[:, -3], y_score=test_predictions)
    logger.info(f"AUC is {auc}")

    all_aucs[epsilon] = auc
    all_coefs[epsilon] = coefs

logger.info(all_aucs)

In [ ]:
keys = sorted(all_aucs.keys())
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)

fig, ax1 = plt.subplots()

ax1.set_xlabel('epsilon')
ax1.set_ylabel('AUC & l_inf norm')
#plt.xscale('log')
ax1.plot(
    keys, [all_aucs[k] for k in keys], label="Mahalanobis"
)
ax1.plot(
    keys, [linfs[k] for k in keys], label="l_inf"
)

ax2 = ax1.twinx()
ax2.set_ylabel("l_2 norm")

ax2.plot(
    keys, [l2s[k] for k in keys], label="l_2", color="green"
)

ax1.legend()
ax2.legend()

## Unsupervised Learning

In [ ]:
from sklearn.svm import OneClassSVM

scaler = StandardScaler()

aucs = dict()
coefs = dict() 

for key in datasets:
    
    ds_train, ds_test = datasets[key]
    
    detector = OneClassSVM(
        tol=1e-5,
        kernel='rbf',
        nu=1e-4
    )
        
    ds_train_scaled = scaler.fit_transform(ds_train.iloc[:, :-3])

    detector.fit(X=ds_train_scaled)
    #coef = list(detector.coef_.flatten())
    #logger.info(f"Coefs of detector {coef}")
    #logger.info(f"Intercept of detector {detector.intercept_}")
    
    ds_test_scaled = scaler.transform(ds_train.iloc[:, :-3])
    
    train_predictions = detector.decision_function(X=ds_train_scaled)
    
    test_predictions = detector.decision_function(X=ds_test_scaled)
    logger.info(f"Average prediction {np.mean(test_predictions)}")
    auc = roc_auc_score(y_true=ds_test.iloc[:, -3], y_score=test_predictions)
    
    logger.info(f"AUC for {key} => {auc}")

    aucs[key] = auc
    #coefs[key] = coef
    
logger.info(f"All AUCS are {aucs}")

In [ ]:
keys = sorted(aucs.keys())
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)

fig, ax1 = plt.subplots()

ax1.set_xlabel('epsilon')
ax1.set_ylabel('AUC & l_inf norm')
#plt.xscale('log')
ax1.plot(
    keys, [aucs[k] for k in keys], label="Mahalanobis"
)
ax1.plot(
    keys, [linfs[k] for k in keys], label="l_inf"
)

ax2 = ax1.twinx()
ax2.set_ylabel("l_2 norm")

ax2.plot(
    keys, [l2s[k] for k in keys], label="l_2", color="green"
)

ax1.legend()
ax2.legend()